In [2]:
!pip install awswrangler

  Using cached awswrangler-3.4.2-py3-none-any.whl.metadata (23 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached awswrangler-3.4.2-py3-none-any.whl (396 kB)
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.

In [3]:
import pandas as pd
import awswrangler as wr
import numpy as np

In [4]:
df = pd.read_parquet("2023-11-14-df-estimacao.parquet.gzip")
df.shape

(259641, 146)

In [5]:
df.head(5)

,id_customer,ds_classif_savings,nr_purchases_online,nr_cdb,nr_bag,flag_mgm,ACADEMIAS,ACOUGUE,ADVOCACIA,APARELHOS_DIVERSOS,...,e_REPAROS,e_RESTAURANTES,e_SEGURO,e_SERVICOS,e_SUPRIMENTOS_PARA_CASA,e_TAXIS,e_TI,e_VAREJISTAS,e_VESTUARIO,e_VIAGEM
0,c82c56ee-8d03-4496-8780-da09cdeb435a,False,True,True,True,True,False,False,False,False,...,True,False,False,False,True,False,True,True,True,False
1,b2459cfa-daee-4c0f-a489-be40733b063b,False,True,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,True,True,False
2,682c7c5c-dd08-47ba-a46f-f1c360662ed0,False,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2b743070-8c64-46ef-a29d-ab5615308dbe,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,63ddbc3f-a917-4a10-8ecc-3872f53f5c01,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


In [6]:
# Contagem da recomendação estimada
col_est = df.filter(like='e_').columns
df['qtd_cat_est'] = df[col_est].sum(axis = 1)

# Contagem real do uso 
col_real = col_est.str.replace("e_","")
df['qtd_cat_real'] = df[col_real].sum(axis = 1)

In [7]:
df.groupby(df['qtd_cat_est']).size()

qtd_cat_est
0     49563
1      5629
2     11076
3      8767
4     13247
      ...  
59      337
60      148
61       75
62       55
63        8
Length: 64, dtype: int64

In [8]:
df.groupby(df['qtd_cat_real']).size()

qtd_cat_real
0     17468
1     17946
2     13583
3      4853
4      5817
5      6970
6      8109
7      9020
8      9875
9     10244
10    10835
11    11031
12    11205
13    11410
14    11522
15    11521
16    11121
17    10484
18     9996
19     9387
20     8537
21     7638
22     6589
23     5534
24     4717
25     3864
26     2902
27     2261
28     1639
29     1214
30      833
31      537
32      367
33      231
34      158
35       92
36       47
37       29
38       23
39       14
40        5
41        5
42        3
43        2
45        1
46        1
56        1
dtype: int64

In [9]:
df.columns

Index(['id_customer', 'ds_classif_savings', 'nr_purchases_online', 'nr_cdb',
       'nr_bag', 'flag_mgm', 'ACADEMIAS', 'ACOUGUE', 'ADVOCACIA',
       'APARELHOS_DIVERSOS',
       ...
       'e_SEGURO', 'e_SERVICOS', 'e_SUPRIMENTOS_PARA_CASA', 'e_TAXIS', 'e_TI',
       'e_VAREJISTAS', 'e_VESTUARIO', 'e_VIAGEM', 'qtd_cat_est',
       'qtd_cat_real'],
      dtype='object', length=148)

In [10]:
def query(query, username):
    """Retorna df da consulta ao banco de dados"""
    
    from datetime import datetime
    dt = datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')
    path = f"s3://will-sdx-ml-platform-sagemaker-transient/{username}/{dt}"

    df = wr.athena.read_sql_query(query, 
    database=None, 
    ctas_approach=False, 
    s3_output=path)

    return df

In [11]:
query_string = '''

with eventos as (
	select 
	id_customer
	, count(distinct dt_event) as nr_acessos
	from customer_curated_zone.ca_analitico_frontend
	where lower(ds_event_name) <> 'login_knownuser'
	and lower(ds_event_name) <> 'insert_textfield_password_loginknownuser'
	and lower(ds_event_name) <> 'home'
	and lower(ds_event_name) <> 'click_box_tracking_card_home'
	and cast(dt_event as DATE) >= date('2023-01-01')
	and cast(dt_event as DATE) < date('2023-11-14')
	group by 1
)
, rentabilidade_conta as (
select 
id_customer
, sum(vl_rentabilidade_min) as rentabilidade_conta
from customer_curated_zone.ca_model_rent_conta_simples
where ds_yearmonth_rent_calc = '2023-10'
group by 1
)
, rentabilidade_credito as (
select 
id_customer
, sum(ltv) as rentabilidade_credito
, sum(rec_servicos + rec_itr + rec_recarga) as receitas
from public.rentabilidade_cartoes_isabelle_dias
where mesref = '2023-10'
group by 1
)
select
c.id_customer
, nr_acessos
, rentabilidade_credito
, receitas
, rentabilidade_conta
from rentabilidade_credito c
left join eventos e on (e.id_customer = c.id_customer)
left join rentabilidade_conta co on (co.id_customer = c.id_customer)

'''

In [12]:
df_2 = query(query_string, 'flavia-costa')

In [14]:
df_2.shape

(4269312, 5)

In [18]:
df.dtypes

id_customer            string[python]
ds_classif_savings               bool
nr_purchases_online              bool
nr_cdb                           bool
nr_bag                           bool
                            ...      
e_VAREJISTAS                     bool
e_VESTUARIO                      bool
e_VIAGEM                         bool
qtd_cat_est                     int64
qtd_cat_real                    int64
Length: 148, dtype: object

In [19]:
df_2.dtypes

id_customer              string[python]
nr_acessos                        Int64
rentabilidade_credito           float64
receitas                        float64
rentabilidade_conta             float64
dtype: object

In [20]:
df_3 = df.join(df_2.set_index('id_customer'), on = 'id_customer', how = 'left')

In [21]:
df_3.head()

,id_customer,ds_classif_savings,nr_purchases_online,nr_cdb,nr_bag,flag_mgm,ACADEMIAS,ACOUGUE,ADVOCACIA,APARELHOS_DIVERSOS,...,e_TI,e_VAREJISTAS,e_VESTUARIO,e_VIAGEM,qtd_cat_est,qtd_cat_real,nr_acessos,rentabilidade_credito,receitas,rentabilidade_conta
0,c82c56ee-8d03-4496-8780-da09cdeb435a,False,True,True,True,True,False,False,False,False,...,True,True,True,False,34,19,187,3.635360,35.46120,NaN
1,b2459cfa-daee-4c0f-a489-be40733b063b,False,True,False,False,False,False,False,False,False,...,False,True,True,False,24,11,63,3.724320,25.33440,NaN
2,682c7c5c-dd08-47ba-a46f-f1c360662ed0,False,True,True,True,False,False,False,False,False,...,False,False,False,False,6,6,26,-2.876000,5.65500,NaN
3,2b743070-8c64-46ef-a29d-ab5615308dbe,True,True,True,True,True,False,False,False,False,...,False,False,False,False,14,14,115,20.058960,6.54820,NaN
4,63ddbc3f-a917-4a10-8ecc-3872f53f5c01,False,True,False,False,True,False,False,False,False,...,False,True,True,False,13,16,17,0.085552,15.91259,NaN


In [22]:
df_3['nr_acessos'].sum()

19833694

In [23]:
df_agg = df_3.groupby('qtd_cat_real') \
.agg({'id_customer':['count'], 'nr_acessos':['sum'], \
      'rentabilidade_credito':['sum'], \
      'receitas':['sum'], \
     'rentabilidade_conta':['sum']
     })

In [24]:
df_agg

,id_customer,nr_acessos,rentabilidade_credito,receitas,rentabilidade_conta
,count,sum,sum,sum,sum
qtd_cat_real,,,,,
0,17468,110420,-2.000000,0.00000,-298646.067680
1,17946,139733,-259.540496,199.88443,-15719.257890
2,13583,142757,20458.874047,7595.14075,-38594.644908
3,4853,118764,15862.039800,22494.24522,-13254.419168
4,5817,191425,21715.629055,35352.50321,-12739.073591
5,6970,301860,52539.500618,46573.08506,-10059.871883
6,8109,410771,71172.929215,64546.70378,-4117.157691
7,9020,504846,73431.665281,82151.64423,792.525210


In [25]:
df_agg.to_csv('simulacao_recom.csv')